## Tool to clean the MP keywords where possible

In [1]:
#imports
import pandas as pd #to manage dataframes
from spellchecker import SpellChecker
import re
import requests
import json
from IPython.display import JSON

In [2]:
#Here is a csv with all MP keywords. First step is to import that list as a dataframe
df = pd.read_csv("MP Keywords.csv")
display(df)

,ID,label,ACTION,comment,mp browse,code,uri,notation,definition,map-to
0,2277,Swedish,map to language vocabulary,NaN,https://marketplace.sshopencloud.eu/search?f.k...,%C2%A0+Swedish,https://vocabs.sshopencloud.eu/vocabularies/ss...,NaN,NaN,NaN
1,15,...),delete,NaN,https://marketplace.sshopencloud.eu/search?f.k...,...%29,https://vocabs.sshopencloud.eu/vocabularies/ss...,NaN,NaN,NaN
2,16,.NET,move to tools/services,NaN,https://marketplace.sshopencloud.eu/search?f.k...,.NET,https://vocabs.sshopencloud.eu/vocabularies/ss...,.NET,NaN,NaN
3,11,(Meta)data,keep it in keyword vocab,NaN,https://marketplace.sshopencloud.eu/search?f.k...,open_parenthesismetaclose_parenthesisdata,https://vocabs.sshopencloud.eu/vocabularies/ss...,NaN,NaN,NaN
4,12,(Quantitative) survey research,keep it in keyword vocab,NaN,https://marketplace.sshopencloud.eu/search?f.k...,open_parenthesisquantitativeclose_parenthesis_...,https://vocabs.sshopencloud.eu/vocabularies/ss...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2272,2273,youtube,NaN,NaN,https://marketplace.sshopencloud.eu/search?f.k...,youtube,https://vocabs.sshopencloud.eu/vocabularies/ss...,NaN,NaN,NaN
2273,667,Zeta,NaN,NaN,https://marketplace.sshopencloud.eu/search?f.k...,Zeta,https://vocabs.sshopencloud.eu/vocabularies/ss...,Zeta,NaN,NaN
2274,2274,zip,NaN,NaN,https://marketplace.sshopencloud.eu/search?f.k...,zip,https://vocabs.sshopencloud.eu/vocabularies/ss...,NaN,NaN,NaN
2275,2275,zooming,NaN,NaN,https://marketplace.sshopencloud.eu/search?f.k...,zooming,https://vocabs.sshopencloud.eu/vocabularies/ss...,zooming,NaN,NaN


## Step 2: Remove all non-words

In [51]:
def has_letter(ID):
    target_row = df.loc[df['ID'] == ID]
    label = target_row['label'].item()
    if any(char.isalpha() for char in str(label)) == False:
        print ("Delete for label "+str(label))
        target_row.loc[:, ('ACTION')] = "delete"

In [52]:
df['ID'].apply(has_letter)

Delete for label ...)


0       None
1       None
2       None
3       None
4       None
        ... 
2272    None
2273    None
2274    None
2275    None
2276    None
Name: ID, Length: 2277, dtype: object

## Step 3: remove stop words

In [60]:
def remove_stop_words(ID):
    target_row = df.loc[df['ID'] == ID]
    label = target_row['label'].item()
    stop_words = ["none", "yes", "no"] #expand
    if str(label).lower() in stop_words:
        print ("Delete for label "+str(label))
        target_row.loc[:, ('ACTION')] = "delete"

In [61]:
df['ID'].apply(remove_stop_words)

Delete for label none
Delete for label yes


0       None
1       None
2       None
3       None
4       None
        ... 
2272    None
2273    None
2274    None
2275    None
2276    None
Name: ID, Length: 2277, dtype: object

## Step 4: Check spelling

In [110]:
def check_spelling(ID): #pretty much unusable at the moment -> greek corrected to green -.-
    target_row = df.loc[df['ID'] == ID]
    label = target_row['label'].item()
    word_list = list(str(label).split(" ")) #we need to convert the string to a list and pass as list items
    #word_list = ['archealogical', 'context'] #test
    misspelled = spell.unknown(word_list)
    for i, word in enumerate(misspelled):
        # Get the one `most likely` answer
        corrected_word = spell.correction(word)
        print(f"Corrected {word} to {corrected_word}")
        word_list[i] = corrected_word

    print("Updated Keyword", word_list)

In [111]:
check_spelling(745)

Updated Keyword ['archaeological', 'context']


In [ ]:
df['ID'].apply(check_spelling)

## Step 5: Check if the keyword is found in a vocab

In [6]:
keyword  = "african-history"
api_url = "https://marketplace-api.sshopencloud.eu/api/concept-search?q="+str(keyword)+"&f=f.candiate%3Dfalse&advanced=true&order=score"
response = requests.get(api_url)
#JSON(response.json()) #this is so nice, remember to always use this!!
# now we always find the keyword in the keywords vocab, so we need to filter the list
data = response.json()

# Create a new list excluding items with vocabulary code 'sshoc-keyword'
filtered_concepts = [i for i in data["concepts"] if i['vocabulary']['code'] != 'sshoc-keyword']

# assign back to data['concepts']
data["concepts"] = filtered_concepts
JSON(data["concepts"]) #so nice, always use this

<IPython.core.display.JSON object>